In [ ]:
pip install requests

In [ ]:
pip install geopy

In [ ]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [ ]:
# Load transaction data
dataset = pd.read_csv("retail_sales_dataset.csv")

# Creating store data which can be merged with the dataset to get a whole dataset we will be working on.
store_data = pd.DataFrame({
    "Store_ID": ["S001", "S002", "S003", "S004", "S005", "S006", "S007", "S008", "S009", "S010"],
    "Location": ["New York", "Los Angeles", "Chicago", "Texas", "Atlanta", "Miami", "Colorado", "San Diego", "Oregon", "New Orleans"],
    "Size": ["Large", "Medium", "Small", "Large", "Medium", "Large", "Small", "Large", "Small", "Large"],
    "Category": ["Flagship", "Outlet", "Online", "Outlet", "Flagship", "Online", "Outlet", "Flagship", "Outlet", "Online"]
})

# Randomly assigning the store data to original dataset and then merging it to get the new dataset.
dataset["Store_ID"] = random.choices(store_data["Store_ID"], k=len(dataset))
enriched_data = dataset.merge(store_data, on="Store_ID")
enriched_data.to_csv("dataset1.csv", index=False)
enriched_data.head()

In [ ]:
# Data Cleaning Steps
data = pd.read_csv("dataset1.csv")

data.isnull().sum() #Checking for any null values

data.duplicated().sum() #Checking for duplicates

print("Invalid Dates:", data['Date'].isnull().sum()) #Invalid dates if any 

data['Date'] = pd.to_datetime(data['Date'], errors='coerce') #Standardize Date Format for easier feature extraction

data['Category'] = data['Category'].str.strip().str.lower() #Normalize columns 'location' & 'categories'
data['Location'] = data['Location'].str.strip().str.title()


# Boxplot to identify outliers in 'Total Amount' and also cap outliers
sns.boxplot(x=data['Total Amount'])
plt.title('Outliers in Total Amount')
plt.show()
q1, q99 = data['Total Amount'].quantile([0.01, 0.99])
data['Total Amount'] = np.clip(data['Total Amount'], q1, q99)

#Adding new features on date columns to reveal insights and help identify sales over time
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Weekday'] = data['Date'].dt.day_name()
data['Is_Weekend'] = data['Weekday'].isin(['Saturday', 'Sunday'])

#Calculating 'Sales Per Customer' and 'Repeat Purchase Columns' to be used
data['onetime_customers'] = data['Customer ID'] = 1 #Since we don't have NULL values we don't need OR condition.
data['repeat_customers'] = data['Customer ID'] > 1

In [ ]:
data.head()

In [ ]:
#Integrating the weather data into the dataset using the OpenWeatherMap API:

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="store_locator")

#Getting lat/lon for each unique location
locations = data['Location'].unique()
location_coords = {}

for loc in locations:
    try:
        geocode = geolocator.geocode(loc)
        location_coords[loc] = {'lat': geocode.latitude, 'lon': geocode.longitude}
    except:
        print(f"Could not find coordinates for {loc}")
        location_coords[loc] = {'lat': None, 'lon': None}

coords_df = pd.DataFrame.from_dict(location_coords, orient='index').reset_index() # Convert to a DataFrame for easier merging
coords_df.columns = ['Location', 'latitude', 'longitude']

data = data.merge(coords_df, on='Location', how='left') # Merge coordinates into the sales dataset

In [ ]:
#Use the OpenWeather API to fetch historical weather data.
import requests
from datetime import datetime

# Your OpenWeatherMap API key
API_KEY = "e77044813be42ef916e030f8c020a8c60"

# Function to fetch weather data
def fetch_weather(lat, lon, date, api_key):
    try:
        # Convert date to UNIX timestamp
        date_object = datetime.strptime(date, '%Y-%m-%d').replace(hour=12, minute=0, second=0) # Convert string to datetime and set time to noon
        unix_timestamp = int(date_object.timestamp())
        
        # API endpoint for historical weather
        url = "https://history.openweathermap.org/data/2.5/history/city?lat={}&lon={}&dt={}&appid={}&units=metric".format(lat, lon, unix_timestamp, API_KEY)
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            # Extract key weather data
            weather = {
                "temperature": data["current"]["temp"],
                "weather_condition": data["current"]["weather"][0]["description"],
            }
            return weather
        else:
            return {"temperature": None, "weather_condition": None}
    except Exception as e:
        print(f"Error fetching weather for {lat}, {lon} on {date}: {e}")
        return {"temperature": None, "weather_condition": None}

# Loop through sales data and add weather data
weather_data = []

for index, row in data.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    date = row['Date'].strftime('%Y-%m-%d') 
    weather = fetch_weather(lat, lon, date, API_KEY)
    weather_data.append(weather)

# Convert weather data to DataFrame and merge with sales data
weather_df = pd.DataFrame(weather_data)
data = pd.concat([data.reset_index(drop=True), weather_df], axis=1)

In [ ]:
CALENDARIFIC_API_KEY = "VMXn8LmZ3sBgo4COl1oLfykap9QR5WMp"

def fetch_holidays(year, country): 
    url = f"https://calendarific.com/api/v2/holidays"
    params = {
        "api_key": CALENDARIFIC_API_KEY,
        "country": country, 
        "year": year,
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        holidays = response.json()["response"]["holidays"]
        holiday_list = []
        for holiday in holidays:
            holiday_list.append({
                "date": holiday["date"]["iso"],
                "name": holiday["name"],
                "type": holiday["type"][0] if holiday["type"] else None
            })
        return pd.DataFrame(holiday_list)
    else:
        print(f"Failed to fetch holidays for {year}")
        return pd.DataFrame()

# Fetch holidays for 2023 and 2024
holidays_2023 = fetch_holidays(2023, "US")  
holidays_2024 = fetch_holidays(2024, "US")
holidays = pd.concat([holidays_2023, holidays_2024], ignore_index=True)

holidays['date'] = pd.to_datetime(holidays['date'])

data['is_holiday'] = data['Date'].isin(holidays['date']) # Mark holidays in the sales data
data['holiday_name'] = data['Date'].map(
    lambda x: holidays.loc[holidays['date'] == x, 'name'].values[0] if x in holidays['date'].values else None
)

In [ ]:
# Count missing values in each column
print(data.isnull().sum())

In [ ]:
#Saving the new updated, cleaned dataset.
data["Transaction ID"] = range(1, len(dataset) + 1) 
data.to_csv("dataset2.csv", index=False)
data.head()